# <font color="maroon">Chapter 9: ROMANCE Analysis of Backbone Motion</font>

Some text about [ROMANCE](https://doi.org/10.1016/j.jmro.2022.100045) analysis.

## Setup and data downloads
Since we've learned now how pyDR is organized and allows us to manage larger data sets, we'll now use the full project functionality.

In [0]:
# SETUP pyDR
!git clone https://github.com/alsinmr/pyDR.git

In [2]:
#Imports
import pyDR

In [3]:
#Download MD files if running in Colab (hidden on webpage)
import sys
if 'google.colab' in sys.modules:
    from gdown import download
    ID='1xgp5_BVeCh6Weu4tnsl1ToRZR49eM5hW'   #HETs xtc file
    #Just grab this string out of the google drive share link 'https://drive.google.com/file/d/1xgp5_BVeCh6Weu4tnsl1ToRZR49eM5hW/view?usp=share_link'
    download(f'https://drive.google.com/uc?id={ID}','/content/pyDR/examples/HETs15N/backboneB.xtc')

In [4]:
# Project Creation and File loading
proj=pyDR.Project()

sel=pyDR.MolSelect(topo='pyDR/examples/HETs15N/backboneB.pdb',
                   traj_files='pyDR/examples/HETs15N/backboneB.xtc',
                   project=proj)  #Selection object

# Specify the bond select to analyze for MD
sel.select_bond('N')

sel.traj.step=10  #Take every tenth point for MD calculation (set to 1 for more accurate calculation)
pyDR.Defaults['ProgressBar']=False #Turns of the Progress bar (screws up webpage compilation)

## Run the frame analysis

### Setup the frame analysis

In [5]:
from numpy import nan
bsheets=[(226,228),(230,234),(236,240),(242,245),
         (262,264),(266,270),(272,276),(278,281)]

# Define second reference frame (beta sheets)
sel0=[sel.uni.select_atoms(f'resid {bsheet[0]}-{bsheet[1]}') for bsheet in bsheets]

fo=pyDR.Frames.FrameObj(sel)
fo.tensor_frame(sel1=1,sel2=2)  #Define the frame of the interaction (dipole tensor)
fo.new_frame(Type='peptide_plane',resids=sel.label) #Define first reference frame
fo.new_frame(Type='superimpose',sel=sel0) #Define second reference frame

### Import the frames, process

In [6]:
fo.frames2data()  #Process frames, send to data
proj['Frames'].detect.r_auto(7)  #Prepare detectors
proj['Frames'].fit().opt2dist(rhoz_cleanup=True)  #Fit and optimize fit

KeyboardInterrupt: 

### Plot the results

First, we validate the frame analysis quality by comparing the product of the individual correlation functions to the total correlation function

In [ ]:
proj.plot_obj.close('all')
proj['opt_fit']['Frames']['Direct'].plot(style='bar')
proj['opt_fit']['Frames']['Product'].plot().fig.set_size_inches([8,12])

Plot the results for the Principal Axis System to the peptide plane (H–N librational motion)

In [ ]:
proj.close_fig('all')
proj['opt_fit']['PAS>peptide_plane'].plot(style='bar').fig.set_size_inches([8,12])
for a in proj.plot_obj.ax[:-1]:a.set_ylim([0,.05])

Plot the results for peptide plane to $\beta$-sheet superimposition (peptide plane dynamics)

In [ ]:
proj.close_fig('all')
proj['opt_fit']['peptide_plane>superimpose'].plot(style='bar').fig.set_size_inches([8,12])
for a in proj.plot_obj.ax[:-1]:a.set_ylim([0,.2])

Plot the results for $\beta$-sheet superimposition to lab frame ($\beta$-sheet dynamics

In [ ]:
proj.close_fig('all')
proj['opt_fit']['superimpose>LF'].plot(style='bar').fig.set_size_inches([8,12])
for a in proj.plot_obj.ax[:-1]:a.set_ylim([0,.05])